In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
# using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo
using Distributions

  Activating project at `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Project.toml`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Manifest.toml`


In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
# import piano model
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
data = init_data(model)

MuJoCo Data object

Model info

In [4]:
println("number of geoms: ",model.ngeom)
println("number of controls: ",model.nu)
println("number of states: ",model.nv+model.nq)
forward!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
finger_geom_indices = [42,50,58,68,77]
"""
index finger:
    name:
        rh_shadow_hand//unnamed_geom_25
        rh_shadow_hand//unnamed_geom_26
    index:
        42
        43
    range:
        x:
        y:-0.6-0.5
        z:0-0.3

middle finger:
    name:
        rh_shadow_hand//unnamed_geom_33
        rh_shadow_hand//unnamed_geom_34
    index:
        50
        51
ring finger:
    name:
        rh_shadow_hand//unnamed_geom_41
        rh_shadow_hand//unnamed_geom_42
    index:
        58
        59
pinky finger:
    name:
        rh_shadow_hand//unnamed_geom_51
        rh_shadow_hand//unnamed_geom_52
    index:
        68
        69
thumb:
    name:
        rh_shadow_hand//unnamed_geom_60
        rh_shadow_hand//unnamed_geom_61
    index:
        77
        78
""";

number of geoms: 79
number of controls: 7
number of states: 46
Simulation timestep: 0.01
Positions of joints: [0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]
Cartesion position of joints: [0.0 0.0 0.0; -0.127 0.0 0.02; 0.0 -0.094 0.01125; -0.03 -0.08224999999999999 0.026000000000000002; 0.0 -0.07050000000000001 0.01125; 0.0 -0.047 0.01125; -0.03 -0.03474999999999999 0.026000000000000002; 0.0 -0.023499999999999993 0.01125; -0.03 -0.011249999999999996 0.026000000000000002; 0.0 0.0 0.01125; 0.0 0.023499999999999993 0.01125; -0.03 0.035750000000000004 0.026000000000000002; 0.0 0.047000000000000014 0.01125; -0.03 0.05925 0.026000000000000002; 0.0 0.07050000000000001 0.01125; -0.03 0.08275 0.026000000000000002; 0.0 0.094 0.01125; 0.3938251578954367 0.14997497683550026 0.13062742818321735; 0.29337550270087137 0.15000084946805123 0.12910710828129915; 0.3072543776776233 0.15000002300031276 0.1299999783510228; 0.21900000000000

Visualize target finger position

In [5]:
# control indices:
# 1: wrist left right
# 2: wrist up down
# 8, 9, 10: index finger joints
# 21: whole hand left right 
# 22: whole hand front back

In [6]:
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states = zeros(nx, 19)
states_geom = zeros(3, 19)
U = zeros(model.nu)
U[4] = 0.664
U[5] = 1.1
U[2] = 0.21
# U[1] = -0.234
record_t_3 = zeros(nx)
data = init_data(model)
step!(model, data)
println(get_physics_state(model, data))
data = init_data(model)
step!(model, data)
println(get_physics_state(model, data))

step!(model, data)
println(get_physics_state(model, data))

states[:,1] = get_physics_state(model, data)
states_geom[:,1] = data.geom_xpos[finger_geom_indices[1],:]
for t in 1:18
    # println(t,data.geom_xpos[finger_geom_indices[1],:])
    data.ctrl .= U
    step!(model, data)
    
    states[:,t+1] = get_physics_state(model, data)
    states_geom[:,t+1] = data.geom_xpos[finger_geom_indices[1],:]
end
# println(data.geom_xpos[finger_geom_indices[1],:])
reset!(model, data)

data.ctrl .= U
data.qpos .= states[:,1][1:model.nq]
data.qvel .= states[:,1][model.nq+1:end]
forward!(model, data)
# println("hi")
println("hi",data.geom_xpos[finger_geom_indices[1],:])
for t in 1:19
    println(t,states_geom[:,t])
end
# init_visualiser()
# visualise!(model, data, trajectories = [states])

[-0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.0016775113564068205, -0.0003039697527948987, -0.008912216587082428, -4.2341315644610154e-5, -0.042233680988673, 0.004593844064341336, 0.0034952271077786437, 0.0017690537583294683, 0.00045559012633187955, 0.00010943869295629263, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.16775113564068206, -0.03039697527948987, -0.8912216587082428, -0.004234131564461016, -4.2233680988673, 0.45938440643413353, 0.34952271077786434, 0.17690537583294683, 0

## Test DIRCOL: moving index finger left right

In [7]:
function wrapped_mj_step(model, data, x, uk, forward)
    # given the current model and data. set the state and control to the model and perform a forward step
    # data.ctrl .= zeros(model.nu)
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] .= uk
        # set state
        # data.qpos .= x[1:model.nq]
        # data.qvel .= x[(model.nq + 1):end]
        data.qpos .= x[1:model.nq]
        data.qvel .= x[model.nq+1:end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= converted_uk

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    if forward
        forward!(model,data)
    else
        step!(model, data) 
    end

    # return updated state k + 1
    # xkp1 = zeros(model.nq + model.nv) 
    # xkp1 .= get_physics_state(model, data)
    # finger_coordinates = data.geom_xpos[finger_geom_indices,:]
    
    return data.geom_xpos[finger_geom_indices[1],:], get_physics_state(model, data)
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    model = params.model
    data = init_data(model)
    step!(model, data)
    Q, R, R_d = params.Q, params.R, params.R_d

    # stage cost
    cost = 0.0
    xi = Z[idx.x[1]]
    ui = Z[idx.u[1]]

    finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
    cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui
    for i = 2:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        ui_1 = Z[idx.u[i-1]]
        finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
        # @show xi-xg
        # cost += 0.5*(xi[94:96]-xg[94:96])'*Q*(xi[94:96]-xg[94:96])+0.5*ui'*R*ui
        cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui#+0.5*(ui-ui_1)'*R_d*(ui-ui_1)
    end

    # # terminal cost 
    # xf = Z[idx.x[N]]
    # # cost += 0.5*(xf[94:96]-xg[94:96])'*Qf*(xf[94:96]-xg[94:96])
    # finger_geom_xpos, _ = wrapped_mj_step(model, data, xf, 0, false)
    # cost += 0.5*(finger_geom_xpos-xg)'*Qf*(finger_geom_xpos-xg)

    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = init_data(model)
    step!(model, data)
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        _,  xip1_mujoco = wrapped_mj_step(model, data, xi, ui, false)

        c[idx.c[i]] = xip1_mujoco - xip1
    end
    # println(typeof(c))
    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    
    # initial_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[1]], Z[idx.u[1]], true)
    terminal_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[N-1]], Z[idx.u[N-1]] , true)
    

    con_1 =  Z[idx.x[1]] - xic
    
    con_2 =  terminal_pose_index - xg

    return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    # return zeros(eltype(Z), 0)
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    
    # # initial_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[1]], Z[idx.u[1]], true)
    terminal_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[N-1]], Z[idx.u[N-1]] , true)
    

    # con_1 =  Z[idx.x[1]] - xic
    
    cart_z_end =  terminal_pose_index[3]

    # return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    # return zeros(eltype(Z), 0)
    return [cart_z_end]
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [26]:

function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("piano_with_one_shadow_hand_small/scene.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)

    # initiate time and time steps
    dt = 0.01
    tf = 0.1
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost
    # Q size of full states
    # Q = diagm(ones(model.nq + model.nv))
    # R = 0.1*diagm(ones(model.nu))
    # Qf = 10*diagm(ones(model.nq + model.nv))

    # Q = 1*diagm([5,5,10])
    Q = 10*diagm(ones(3))
    R = 0.1*diagm(ones(model.nu))
    R_d = 10*diagm(ones(model.nu))

    idx = create_idx(model.nq + model.nv, model.nu, N)
    # initial and goal states
    step!(model, data)
    xic = get_physics_state(model, data)
    xic_geom = data.geom_xpos[finger_geom_indices[1],:]
    U = zeros(model.nu)
    U[4] = 0.664
    U[5] = 1.1
    U[2] = 0.21
    for t in 1:N-1
        data.ctrl .= U
        step!(model, data)
    end
    # initial and goal state
    xg_physics = get_physics_state(model, data)
    xg = data.geom_xpos[finger_geom_indices[1],:]
    # reset the model and data
    reset!(model, data)
    params = (Q = Q, R = R, R_d = R_d, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model)

    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:N
        for j = 1:model.nq
            # hands
            x_l[idx.x[i][j]] = model.jnt_range[j,1]
            x_u[idx.x[i][j]] = model.jnt_range[j,2]
        end
    end
    for i = 1:(N-1)
        for j = 1:model.nu
            x_l[idx.u[i][j]] = model.actuator_ctrlrange[j,1]
            x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        end
        # for j in [2,5,4]
        #     x_l[idx.u[i][j]] = 0
        #     x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        # end
    end
    

    # inequality constraints
    c_l = [xg[3] - 0.1]
    c_u = [xg[3] + 0.001]
    # c_l = zeros(0)
    # c_u = zeros(0)
    # c_l = -0.01*ones(idx.nx+3+idx.nx*(N-1))
    # c_u = 0.01*ones(idx.nx+3+idx.nx*(N-1))
    # initial guess 
    z0 = randn(idx.nz)
    z0[idx.x[1]] .= xic
    for i = 2:N
        z0[idx.x[i]] .= xic + ((xg_physics-xic)/(N))*(i)
        # z0[idx.x[i]] .= xic
    end
    println(length(robohand_inequality_constraints(params,z0)))
    println(idx.nx)
    for i = 1:N-1
        for j = 1:model.nu
            z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        end
        # for j in [2,5,4]
        #     z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        # end
    end
    # diff type
    # diff_type = :auto 
    diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-3, c_tol = 1e-2, max_iters = 500, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [27]:
X, U, t_vec, params = solve_finger_moving(verbose=true)

1
46
---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:   293184
Number of nonzeros in inequality constraint Jacobian.:      576
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      576
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      323
                     variables with only upper bounds:        0
Total number of equality constraints.................:      509
Total number of inequality constraints..

([[-9.999999973841333e-9, -9.999999995216623e-9, -9.99999997384133e-9, -9.999999973841341e-9, -9.999999995216618e-9, -9.999999973841341e-9, -9.999999995216613e-9, -9.999999973841335e-9, -9.999999973841328e-9, -9.999999995216622e-9  …  -0.16775113564067243, -0.030396975279481788, -0.8912216587082428, -0.004234131564461016, -4.2233680988673, 0.45938440643413353, 0.34952271077786434, 0.17690537583294683, 0.045559012633187956, 0.010943869295629263], [-9.999999967326321e-9, -9.999999994291751e-9, -9.999999967326321e-9, -9.999999967326321e-9, -9.99999999429175e-9, -9.999999967326321e-9, -9.999999994291748e-9, -9.999999967326318e-9, -9.999999967326308e-9, -9.999999994291751e-9  …  -0.07132391608179091, -0.011787507319059663, -0.7838944698148823, 0.03420941967938343, -4.2322255330679415, 0.028056811926118537, 0.17186878339656883, 10.353814751921085, 12.796112067823902, 13.665679422484905], [-9.99999996480255e-9, -9.999999993908323e-9, -9.999999964802545e-9, -9.999999964802547e-9, -9.9999999939

In [28]:
X

11-element Vector{Vector{Float64}}:
 [-9.999999973841333e-9, -9.999999995216623e-9, -9.99999997384133e-9, -9.999999973841341e-9, -9.999999995216618e-9, -9.999999973841341e-9, -9.999999995216613e-9, -9.999999973841335e-9, -9.999999973841328e-9, -9.999999995216622e-9  …  -0.16775113564067243, -0.030396975279481788, -0.8912216587082428, -0.004234131564461016, -4.2233680988673, 0.45938440643413353, 0.34952271077786434, 0.17690537583294683, 0.045559012633187956, 0.010943869295629263]
 [-9.999999967326321e-9, -9.999999994291751e-9, -9.999999967326321e-9, -9.999999967326321e-9, -9.99999999429175e-9, -9.999999967326321e-9, -9.999999994291748e-9, -9.999999967326318e-9, -9.999999967326308e-9, -9.999999994291751e-9  …  -0.07132391608179091, -0.011787507319059663, -0.7838944698148823, 0.03420941967938343, -4.2322255330679415, 0.028056811926118537, 0.17186878339656883, 10.353814751921085, 12.796112067823902, 13.665679422484905]
 [-9.99999996480255e-9, -9.999999993908323e-9, -9.999999964802545e-9, -

In [29]:
U
# U2[4] = 0.664
# U2[5] = 1.1
# U2[2] = 0.21

10-element Vector{Vector{Float64}}:
 [-0.34188401397807416, -0.016210042896148785, 0.03553530388438475, 0.9587512278093648, 1.9868034150979539, -0.08522473943524866, 0.051568638214831215]
 [-0.2606104240354554, -0.05824473462170486, -0.01784311788201275, 0.9344792876852742, 2.2612252370282575, -0.08261521594067084, 0.04563880009331506]
 [-0.23648473534492828, -0.0006921939255330907, -0.07831618083586976, 0.6932327363328021, 1.9978518905956864, -0.08802536102531942, 0.04154079763262194]
 [-0.24181451367752183, 0.0248206071148467, -0.0650699054274476, 0.923612676865817, 1.2036027530076214, -0.07980850855646755, 0.038110401318401474]
 [-0.28223274279125904, 0.1308347801268202, -0.04093374620235157, 1.068387536477431, 1.6728100499333278, -0.07931597684696527, 0.032827045828510246]
 [-0.3732360961812092, -0.024567243313991908, -0.057900645299349185, 0.6940806649807675, 1.4505455658279751, -0.07822984758623852, 0.008249537418395196]
 [-0.391672234520943, 0.1627547856219715, 0.082281686065891

In [30]:
# instantiate model and data
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
data = init_data(model)

# reset the model and data j in [2,5,4]
        #     z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        # end
dt = 0.01
tf = 0.1
t_vec = 0:dt:tf
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
reset!(model, data)
step!(model, data)
states[:,1] = get_physics_state(model, data)
for t in 1:N-1
    data.ctrl .= U[t][:]
    step!(model, data)
    states[:,t+1] = get_physics_state(model, data)
end
println("end using controls:",data.geom_xpos[finger_geom_indices[1],:])
states3 = zeros(nx, N)
for t in 1:N
    states3[:,t] = X[t][:]
end

reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states2 = zeros(nx, N)
states2[:,1] = get_physics_state(model, data)
U2 = zeros(model.nu)
U2[4] = 0.664
U2[5] = 1.1
U2[2] = 0.21
for t in 1:N-1
    data.ctrl .= U2
    step!(model, data)
    states2[:,t+1] = get_physics_state(model, data)
end
println(data.geom_xpos[finger_geom_indices[1],:])

end using controls:[0.021263602298859607, 0.09997382657396016, 0.061741627881559916]
[0.025231893934994348, 0.07195922272699179, 0.05643644998958352]


In [31]:
init_visualiser()
visualise!(model, data, trajectories = [states])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [32]:
init_visualiser()
visualise!(model, data, trajectories = [states2])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [33]:
init_visualiser()
visualise!(model, data, trajectories = [states3])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.
